# SO-100 Teleoperation
Run cells top to bottom. Ports and cameras are auto-detected so it works regardless of USB enumeration order.

## 1. Detect Ports & Cameras
Scans all `/dev/ttyACM*` and `/dev/ttyUSB*` ports, reads motor voltage to identify leader (~5V USB) vs follower (~12V PSU).
Reads `/sys/class/video4linux/` to identify wrist (Innomaker) and top (other USB) cameras, ignoring the laptop webcam.

In [ ]:
import glob
import scservo_sdk as scs
from pathlib import Path

# ── Motor port detection ─────────────────────────────────────────────
VOLTAGE_ADDR = 62  # Present_Voltage, 1 byte, unit = 0.1V

# Find all serial ports dynamically
ports = sorted(glob.glob("/dev/ttyACM*") + glob.glob("/dev/ttyUSB*"))
if not ports:
    print("✗ No serial ports found! Are the arms plugged in?")

leader_port = None
follower_port = None

for port_path in ports:
    port = scs.PortHandler(port_path)
    if not port.openPort():
        print(f"  {port_path}: CANNOT OPEN")
        continue
    port.setBaudRate(1000000)
    ph = scs.PacketHandler(0)

    data, comm, _ = ph.readTxRx(port, 1, VOLTAGE_ADDR, 1)
    port.closePort()

    if comm != 0:
        # Not a motor controller on this port
        continue

    volts = data[0] / 10.0
    if volts > 8:
        label = "FOLLOWER (~12V)"
        follower_port = port_path
    else:
        label = "LEADER  (~5V)"
        leader_port = port_path

    print(f"  {port_path}: {volts:.1f}V  →  {label}")

if leader_port and follower_port:
    print("✓ Ports detected OK!")
else:
    print("✗ ERROR: Could not find both arms! Are they plugged in?")

# ── Camera detection ─────────────────────────────────────────────────
# Patterns to IGNORE (laptop integrated cameras)
IGNORE_PATTERNS = ["webcam", "integrated", "laptop"]
# Pattern for the Innomaker wrist camera
WRIST_PATTERN = "innomaker"

wrist_cam_index = None
top_cam_index = None
seen_names = set()

print()
sysfs = Path("/sys/class/video4linux")
for dev in sorted(sysfs.iterdir(), key=lambda d: d.name):
    name_file = dev / "name"
    if not name_file.exists():
        continue

    cam_name = name_file.read_text().strip()

    # Skip duplicate entries (each physical camera creates 2 /dev/video* nodes)
    if cam_name in seen_names:
        continue
    seen_names.add(cam_name)

    # Extract the integer index from e.g. "video2"
    idx = int(dev.name.replace("video", ""))
    name_lower = cam_name.lower()

    # Skip laptop / integrated cameras
    if any(p in name_lower for p in IGNORE_PATTERNS):
        print(f"  video{idx}: {cam_name}  →  SKIP (laptop)")
        continue

    # Classify
    if WRIST_PATTERN in name_lower:
        wrist_cam_index = idx
        print(f"  video{idx}: {cam_name}  →  WRIST")
    elif top_cam_index is None:
        top_cam_index = idx
        print(f"  video{idx}: {cam_name}  →  TOP")
    else:
        print(f"  video{idx}: {cam_name}  →  SKIP (extra)")

if wrist_cam_index is not None and top_cam_index is not None:
    print("✓ Both cameras detected!")
elif wrist_cam_index is not None:
    print("✓ Wrist camera detected (no top camera)")
elif top_cam_index is not None:
    print("✓ Top camera detected (no wrist camera)")
else:
    print("⚠ No cameras detected")

## 2. Choose Mode and Run

In [ ]:
# Auto-select mode based on detected cameras (override below if needed)
if wrist_cam_index is not None and top_cam_index is not None:
    MODE = "top_wrist"
elif wrist_cam_index is not None:
    MODE = "wrist_only"
elif top_cam_index is not None:
    MODE = "top_only"
else:
    MODE = "basic"

# Manual override — uncomment ONE to force a mode:
#MODE = "basic"
#MODE = "wrist_only"
#MODE = "top_only"
#MODE = "top_wrist"

# Build camera CLI flags from detected indices
cam_parts = []
if MODE in ("top_only", "top_wrist") and top_cam_index is not None:
    cam_parts.append(f"top: {{type: opencv, index_or_path: {top_cam_index}, width: 640, height: 480, fps: 30}}")
if MODE in ("wrist_only", "top_wrist") and wrist_cam_index is not None:
    cam_parts.append(f"wrist: {{type: opencv, index_or_path: {wrist_cam_index}, width: 640, height: 480, fps: 30}}")

if cam_parts:
    camera_flag = f"--robot.cameras='{{ {', '.join(cam_parts)} }}' --display_data=true"
else:
    camera_flag = ""

assert leader_port and follower_port, "Run cell 1 first! Ports not detected."

cmd = (
    f"lerobot-teleoperate"
    f" --robot.type=so100_follower --robot.port={follower_port} --robot.id=my_follower"
    f" --teleop.type=so100_leader --teleop.port={leader_port} --teleop.id=my_leader"
    f" {camera_flag}"
)

print(f"Mode: {MODE}")
if top_cam_index is not None:
    print(f"  top   → video{top_cam_index}")
if wrist_cam_index is not None:
    print(f"  wrist → video{wrist_cam_index}")
print(f"\nCommand:\n\n{cmd}\n")

In [ ]:
# Run teleoperation (Ctrl+C or interrupt kernel to stop)
import subprocess, shlex, sys

print("Starting teleoperation... (interrupt kernel to stop)\n")
process = subprocess.Popen(shlex.split(cmd), stdout=sys.stdout, stderr=sys.stderr)
process.wait()

## 3. Record Episodes
Uses `lerobot-record` with the same auto-detected ports and cameras. Set your dataset name and task description below.

In [ ]:
# ── Recording settings ────────────────────────────────────────────────
REPO_ID = "umbra-robotics/so100_pick_cube"   # HuggingFace dataset repo
TASK = "Pick up the cube and place it in the bin"
NUM_EPISODES = 1
EPISODE_TIME_S = 60     # max seconds per episode
RESET_TIME_S = 10       # seconds between episodes to reset the scene
FPS = 30
PUSH_TO_HUB = False     # set True to upload to HuggingFace Hub

assert leader_port and follower_port, "Run cell 1 first! Ports not detected."

record_cmd = (
    f"lerobot-record"
    f" --robot.type=so100_follower --robot.port={follower_port} --robot.id=my_follower"
    f" --teleop.type=so100_leader --teleop.port={leader_port} --teleop.id=my_leader"
    f" {camera_flag}"
    f" --dataset.repo_id={REPO_ID}"
    f" --dataset.single_task='{TASK}'"
    f" --dataset.num_episodes={NUM_EPISODES}"
    f" --dataset.episode_time_s={EPISODE_TIME_S}"
    f" --dataset.reset_time_s={RESET_TIME_S}"
    f" --dataset.fps={FPS}"
    f" --dataset.push_to_hub={str(PUSH_TO_HUB).lower()}"
)

print(f"Recording {NUM_EPISODES} episode(s) → {REPO_ID}")
print(f"Task: {TASK}")
print(f"Episode: {EPISODE_TIME_S}s, reset: {RESET_TIME_S}s, fps: {FPS}")
print(f"\nCommand:\n\n{record_cmd}\n")

In [ ]:
# Run recording (interrupt kernel to stop)
import subprocess, shlex, sys

print("Starting recording... (interrupt kernel to stop)\n")
process = subprocess.Popen(shlex.split(record_cmd), stdout=sys.stdout, stderr=sys.stderr)
process.wait()